# 1-D Convolutional Networks for Phoneme Recognition
Architecture inspired by https://arxiv.org/pdf/1707.01836.pdf

In [230]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from scipy import signal
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import sys
sys.path.append('../../')

from speech2phone.preprocessing.TIMIT.phones import get_data, get_phones, phones

### Load Data

In [202]:
def resample(data, y):
    """Resample audio to 800 points."""
    return signal.resample(data, 800), y

def resample512(data, y):
    return signal.resample(data, 512), y

In [216]:
import os
print(os.getcwd())
print(os.path.abspath('../TIMIT/TIMIT'))

audio, labels = get_data(preprocessor=resample, batch_preprocess=False, TIMIT_root='../TIMIT/TIMIT', padding=500)
phonemes = get_phones(labels)
n_phones = len(phones)

/mnt/pccfs/backed_up/jaredtn/speech2phone/experiments
/mnt/pccfs/backed_up/jaredtn/speech2phone/TIMIT/TIMIT
Loading train/resample/500 set from cache... done.


In [232]:
audio_tensor = torch.Tensor(audio).unsqueeze(1)
labels_tensor = torch.Tensor(labels)
print(audio_tensor.shape)
print(labels_tensor.shape)

torch.Size([132810, 1, 800])
torch.Size([132810])


### Train/Test Split

In [233]:
full_dataset = TensorDataset(audio_tensor, labels_tensor) # Dataset requires same batch dimension
train_size = int(0.9 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

### Train the Model

In [ ]:
class ConvNet1D(nn.Module):
    def __init__(self, num_inputs, num_outputs, kernel_size=15, stride=1, padding=7, conv_dropout=0.2):
        super().__init__()
        self.n_channels = 64
        self.n_embedding = self.n_channels * 25
        print("n_embedding: {}".format(self.n_embedding))
        
        # input (N, 1, 800) - no initial dropout before residual connections
        self.init = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=self.n_channels, kernel_size=kernel_size, stride=1, padding=padding),
            nn.BatchNorm1d(self.n_channels),
            nn.ReLU()
        )
        
        self.pool4 = nn.AvgPool1d(kernel_size=4)
        self.pool2 = nn.AvgPool1d(kernel_size=2)
        
        # (N, 128, 800)
        self.downsample4 = nn.Sequential(
            nn.Conv1d(in_channels=self.n_channels, out_channels=self.n_channels, kernel_size=kernel_size, stride=1, padding=padding),
            nn.BatchNorm1d(self.n_channels),
            nn.ReLU(),
            nn.Dropout(conv_dropout),
            nn.Conv1d(in_channels=self.n_channels, out_channels=self.n_channels, kernel_size=kernel_size, stride=4, padding=padding)
        )
        
        # (N, 128, 200)
        
        # apply lots of pooling
        
        # (N, 128, 25)
    
        n_res_blocks = 3
        channel_lengths = [
            self.n_channels * (2 ** (i // 4))
            for i in range(n_res_blocks+1)
        ]
        channel_lengths = [self.n_channels for i in range(n_res_blocks+1)]
#         print(channel_lengths)
        self.convs = nn.ModuleList([nn.Sequential(
            nn.BatchNorm1d(channel_lengths[i]),
            nn.ReLU(),
            nn.Dropout(conv_dropout),
            nn.Conv1d(in_channels=channel_lengths[i], out_channels=channel_lengths[i], kernel_size=kernel_size, stride=1, padding=padding),
            
            nn.BatchNorm1d(channel_lengths[i]),
            nn.ReLU(),
            nn.Dropout(conv_dropout),
            nn.Conv1d(in_channels=channel_lengths[i], out_channels=channel_lengths[i+1], kernel_size=kernel_size, stride=2, padding=padding),
        ) for i in range(n_res_blocks)])
        
        # (N, 128, 25)
        self.predense = nn.Sequential(
            nn.BatchNorm1d(channel_lengths[-1]),
            nn.ReLU()
        )
        self.dense = nn.Linear(self.n_embedding, num_outputs)
    
    def forward(self, x):
        x = self.init(x)
        x = self.pool4(x) + self.downsample4(x)
        
        for conv in self.convs:
            x = self.pool2(x) + conv(x)
#             print(x.shape)
                  
        x = self.predense(x)
        x = x.view(-1, self.n_embedding)
        x = self.dense(x)

        return x
    


model = ConvNet1D(num_inputs=800, num_outputs=61).cuda()
train_loader = DataLoader(train_dataset, batch_size=1024)
test_loader = DataLoader(test_dataset, batch_size=1024)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, verbose=True)
n_epochs = 50
n_print_every = len(train_loader) - 1

print("Batches per Epoch: {}".format(len(train_loader)))

for epoch in range(n_epochs):
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, y_truth = data
        inputs, y_truth = inputs.cuda(), y_truth.long().cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
#         print("inputs: {}, outputs: {}, labels: {}".format(inputs.shape, outputs.shape, y_truth.shape))
        train_loss = criterion(outputs, y_truth)
        train_loss.backward()
        optimizer.step()
        
        y_pred = torch.argmax(outputs, dim=1)
        train_acc = (y_pred == y_truth).float().mean()

        # print statistics
        running_loss += train_loss.item()
        if i % n_print_every == n_print_every - 1:    # print every 2000 mini-batches
            inputs, y_truth = next(iter(test_loader))
            inputs, y_truth = inputs.cuda(), y_truth.long().cuda()
            outputs = model(inputs)
            test_loss = criterion(outputs, y_truth)
            y_pred = torch.argmax(outputs, dim=1)
            test_acc = (y_pred == y_truth).float().mean()
            scheduler.step(test_acc) # anneal the learning rate if no progress for 5 epochs
            
            print('[%d, %5d] train loss: %.3f test loss: %.3f train acc %.3f test acc: %.3f' %
                  (epoch + 1, i + 1, running_loss / n_print_every, test_loss, train_acc, test_acc))
            running_loss = 0.0
            
                            
print("done")

n_embedding: 1600
Batches per Epoch: 117
[1,   116] train loss: 2.091 test loss: 1.494 train acc 0.505 test acc: 0.519
[2,   116] train loss: 1.393 test loss: 1.216 train acc 0.588 test acc: 0.610
[3,   116] train loss: 1.173 test loss: 1.088 train acc 0.657 test acc: 0.637
[4,   116] train loss: 1.052 test loss: 1.026 train acc 0.680 test acc: 0.653
[5,   116] train loss: 0.978 test loss: 0.970 train acc 0.689 test acc: 0.665
[6,   116] train loss: 0.924 test loss: 0.968 train acc 0.706 test acc: 0.678
[7,   116] train loss: 0.880 test loss: 0.926 train acc 0.688 test acc: 0.688
[8,   116] train loss: 0.846 test loss: 0.921 train acc 0.719 test acc: 0.696
[9,   116] train loss: 0.814 test loss: 0.876 train acc 0.742 test acc: 0.702
[10,   116] train loss: 0.789 test loss: 0.868 train acc 0.737 test acc: 0.709
[11,   116] train loss: 0.765 test loss: 0.885 train acc 0.747 test acc: 0.704
[12,   116] train loss: 0.744 test loss: 0.899 train acc 0.769 test acc: 0.710
[13,   116] train lo

Best results so far: 66.5% after 10 epochs (no dropout, 5 128-channel kernel-size-4 convolutional layers, 4-kernel avgpool).  
Increasing the number of convolution channels from 64 to 128 works quite well, with limited signs of overfitting.
Increasing the learning rate from 1e-4 to 1e-3 boosted things as well (66.2% after 5 epochs).  
I'm experimenting with skip connections; hopefully they'll help!  
Dropout just makes things worse.  
model.eval() makes things worse. That's because of how it interacts with the BatchNorm layer; it uses the stored mean/var instead of computing batch statistics.  

Better results!!! 67.7% after 10 epochs (dropout 0.5, 5 layers of 64-channel 16-size-kernel convolutional layers).  
Removing residual connections gave 68.1% after 10 epochs. Probably just noise though.  

Copying the "Cardiologist-Level Arrhythmia Detection" provided 70.0% accuracy after 10 epochs, with dropout 0.5 and Adam lr 3e-3, 64-channel 16-size-kernel convolutional layers.  
Using ReduceLROnPlateau with patience 2 also helped, as did reducing conv_dropout from 0.5 to 0.2, as in their paper. 70.7% accuracy after 10.
Bumping the Adam learning rate up from 1e-3 to 1e-2, combined with ReduceLROnPlateau, gave us 73.0% accuracy after 10 epochs.  

Large batch sizes work well. batch size 32 converges at 67.7%, batch size 128 converges at 73.0%, batch size 256 converges at 75.7%, batch size 512 converges at 73.8%, batch size 1024 converges at 75.9%.  
Batch size 512 converges significantly slower, taking 12 epochs to cross 70%.

In [ ]:
print([p.numel() for p in model.parameters()]) # Number of parameters in each layer